In [1]:
import os
root_dir = f"{os.getcwd().split('TSM-Bench')[0]}/TSM-Bench"
os.chdir(root_dir)
os.getcwd()

'/home/luca/test/TSM-Bench'

In [2]:
import os
import pandas
import systems

from systems import (clickhouse, questdb , extremedb, druid, timescaledb, influx , monetdb)


In [3]:
def load_query_file(system):
    path = os.path.dirname(system.__file__)
    with open(f'{path}/queries.sql') as file: 
        queries = [line.rstrip() for line in file]
    return queries

In [37]:
# settings
n_sensors = [1]# , 10,20,30,40,50,60,70,80,90,100]
n_stations =  [1]# ,2,3,4,5,6,7,8,9,10] 
time_ranges = ["Minute","Hour","Day","Week","Month"]
dataset = "d1"

n_iter = 4
timeout = 1000

scenarios  = [(sensor, station, time_range) for sensor in n_sensors for station in n_stations for time_range in time_ranges]
scenarios

[(1, 1, 'Minute'),
 (1, 1, 'Hour'),
 (1, 1, 'Day'),
 (1, 1, 'Week'),
 (1, 1, 'Month')]

In [38]:
def load_query_file(system):
    path = os.path.dirname(system.__file__)
    with open(f'{path}/queries.sql') as file: 
                queries = [line.rstrip() for line in file]
    return queries
    

In [39]:
systems = (questdb, clickhouse , extremedb, timescaledb, influx , monetdb)
output_file = "results/parameters.txt"

for system in systems:
    system.launch()
    system_name = system.__file__.split("/")[-2]
    queries = load_query_file(system)
    print(queries)
    for i ,query in enumerate(queries):
        query = query.replace("<db>", dataset)
        for n_s, n_st, time_range in scenarios:
            time , var = system.run_query(query , rangeUnit=time_range , rangeL = 1 , n_s=n_s , n_it=n_iter , n_st = n_st) 
            with open(output_file, "a") as file:
                line = f"{time} , {var}  , {i} , {n_s} , {n_st} , {time_range} , {system_name} \n"
                file.write(line)
    system.stop()

launching questdb
/home/luca/test/TSM-Bench/systems/questdb/run_system.py
changing directory to /home/luca/test/TSM-Bench/systems/questdb
["select ts, id_station, <sid> FROM <db> where id_station in <stid> AND  ts < '<timestamp>' AND ts >  '<timestamp>' - <range>*<rangesUnit>* 1000000L", "select ts, id_station, <sid> FROM <db> where id_station in <stid> AND  ts < '<timestamp>' AND ts >  '<timestamp>' - <range>*<rangesUnit>* 1000000L and <sfilter>;", "SELECT id_station, <avg_s> FROM <db> WHERE  ts < '<timestamp>' AND ts >  '<timestamp>' - <range>*<rangesUnit>* 1000000L AND id_station in <stid> GROUP BY id_station;", "SELECT id_station, ts, <avg_s> FROM <db> WHERE ts < '<timestamp>' AND ts >  '<timestamp>' - <range>*<rangesUnit>* 1000000L AND id_station in <stid> SAMPLE BY 1h;", "SELECT id_station, ts, <avg_s> FROM <db> WHERE  ts < '<timestamp>' AND ts >  '<timestamp>' - <range>*<rangesUnit>* 1000000L AND id_station in <stid> SAMPLE BY 5s FILL(LINEAR) GROUP BY ts, id_station ORDER BY ts;

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 19.84it/s]


/home/luca/test/TSM-Bench/systems/questdb/run_system.py
changing directory to /home/luca/test/TSM-Bench/systems/questdb
/home/luca/test/TSM-Bench/systems/clickhouse/run_system.py
changing directory to /home/luca/test/TSM-Bench/systems/clickhouse
["select time, id_station, <sid> FROM <db> where id_station in <stid> AND time > TIMESTAMP '<timestamp>' - INTERVAL '<range>' <rangesUnit> AND time < TIMESTAMP '<timestamp>';", "select time, id_station, <sid> FROM <db> where id_station in <stid> AND time > TIMESTAMP '<timestamp>' - INTERVAL '<range>' <rangesUnit> AND time < TIMESTAMP '<timestamp>' and <sfilter>;", "SELECT id_station, <avg_s> FROM <db> WHERE time > TIMESTAMP '<timestamp>' - INTERVAL '<range>' <rangesUnit> AND time < TIMESTAMP '<timestamp>' AND id_station in <stid> GROUP BY id_station;", 'SELECT id_station, EXTRACT(YEAR FROM time) AS "year", date_trunc(\'month\', time) AS "month", date_trunc(\'day\', time) AS "day", date_trunc(\'hour\', time) AS "hour", <avg_s> FROM <db> where  t

  0%|                                                                                                                                 | 0/4 [00:00<?, ?it/s]


OperationalError: Code: 62.
DB::Exception: Empty query. Stack trace:

0. DB::Exception::Exception(DB::Exception::MessageMasked&&, int, bool) @ 0x000000000c741d97 in /usr/bin/clickhouse
1. DB::Exception::createDeprecated(String const&, int, bool) @ 0x00000000075c238d in /usr/bin/clickhouse
2. DB::parseQueryAndMovePosition(DB::IParser&, char const*&, char const*, String const&, bool, unsigned long, unsigned long) @ 0x0000000013b0427c in /usr/bin/clickhouse
3. DB::executeQueryImpl(char const*, char const*, std::shared_ptr<DB::Context>, bool, DB::QueryProcessingStage::Enum, DB::ReadBuffer*) @ 0x0000000012492314 in /usr/bin/clickhouse
4. DB::executeQuery(String const&, std::shared_ptr<DB::Context>, bool, DB::QueryProcessingStage::Enum) @ 0x000000001249181c in /usr/bin/clickhouse
5. DB::TCPHandler::runImpl() @ 0x00000000133197ec in /usr/bin/clickhouse
6. DB::TCPHandler::run() @ 0x000000001332bc79 in /usr/bin/clickhouse
7. Poco::Net::TCPServerConnection::start() @ 0x0000000015d0eb14 in /usr/b

In [27]:
for (n_s, n_st, time_range) in list(scenarios):
    print(n_s)

In [28]:
list(scenarios)

[]